In [2]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.append('../../')
from containers import Protein
from score.pairs import PosePair
from benchmark import get_data, load, mcss, add_correct

import matplotlib.pyplot as plt
%matplotlib inline

# Interaction Similarity

In [3]:
prot = Protein('PLK1')
prot.load_docking(['2YAC_lig', '3THB_lig'], load_fp=True)
nat_2yac = prot.docking[prot.lm.st].ligands['2YAC_lig'].poses[11]
ref_2yac = prot.docking[prot.lm.st].ligands['2YAC_lig'].poses[0]
nat_3thb = prot.docking[prot.lm.st].ligands['3THB_lig'].poses[4]
ref_3thb = prot.docking[prot.lm.st].ligands['3THB_lig'].poses[9]
print(nat_2yac.rmsd, ref_2yac.rmsd)
print(nat_3thb.rmsd, ref_3thb.rmsd)

1.55056573462 4.66297357026
1.16843506655 5.40489058198


In [4]:
pp = PosePair(nat_2yac, nat_3thb, None)
for feature in ['contact', 'hbond', 'sb']:
    print(feature)
    print(pp.overlap(feature))
    print(pp._total(feature))
    print(pp.tanimoto(feature))

contact
184.26033778902502
383.71530337012337
0.9196116722893387
hbond
1.7681405479783305
5.59003990144845
0.47547035424588574
sb
1.0
2.0
0.6666666666666666


In [5]:
pp = PosePair(ref_2yac, ref_3thb, None)
for feature in ['contact', 'hbond', 'sb']:
    print(feature)
    print(pp.overlap(feature))
    print(pp._total(feature))
    print(pp.tanimoto(feature))

contact
171.54796451774578
362.8253380283428
0.8927478751582131
hbond
0
3.3725273110445055
0.18613213895521064
sb
0
0
0.5


# MCSS Similarity

In [5]:
data = load('stats21', 'pdb')
data = add_correct(data, thresh = 2.0)
data[(data.index.get_level_values('protein') == 'BRD4') & (data.index.get_level_values('features') == 'mcss')]

combind_rank  \
version helpers mode     features alpha n_ligs family protein ligand                   
stats21 pdb     crystal  mcss     0.50  0      Other  BRD4    3P5O_lig             0   
                                                              3U5J_lig             3   
                                                              3U5L_lig             1   
                                                              3ZYU_lig             0   
                                                              4LYW_lig             0   
                                                              4LZR_lig             0   
                                                              4LZS_lig             2   
                                                              4MR3_lig             0   
                                                              4NUC_lig             0   
                                                              4NUE_lig             0   
                                                              4OGJ_lig             0   
                                                              4QB3_lig             0   
                                                              4WIV_lig             0   
                                                              4XYA_lig             0   
                                                              4Z93_lig             0   
                                                              5ACY_lig             0   
                                  0.75  0      Other  BRD4    3P5O_lig             0   
                                                              3U5J_lig             3   
                                                              3U5L_lig             1   
                                                              3ZYU_lig             0   
                                                              4LYW_lig             0   
                                                              4LZR_lig             0   
                                                              4LZS_lig             2   
                                                              4MR3_lig             0   
                                                              4NUC_lig             0   
                                                              4NUE_lig             0   
                                                              4OGJ_lig             0   
                                                              4QB3_lig             0   
                                                              4WIV_lig             0   
                                                              4XYA_lig             0   
                                                              4Z93_lig             0   
                                                              5ACY_lig             0   
                                  1.00  0      Other  BRD4    3P5O_lig             0   
                                                              3P5O_lig             0   
                                                              3U5J_lig             3   
                                                              3U5J_lig             3   
                                                              3U5L_lig             1   
...                                                                              ...   
                standard mcss     1.25  0      Other  BRD4    4QB3_lig             0   
                                                              4WIV_lig             0   
                                                              4XYA_lig             0   
                                                              4Z93_lig             0   
                                                              5ACY_lig             0   
                                  1.50  0      Other  BRD4    3P5O_lig             0   
                        

In [6]:
prot = Protein('BRD4')
prot.lm.mcss.load_mcss()
for ligand in prot.lm.get_xdocked_ligands(20):
    if ligand == '3U5J_lig': continue
    print(ligand, prot.lm.mcss.get_mcss_size('3U5J_lig', ligand))

3P5O_lig 0.9545454545454546
3U5L_lig 0.9545454545454546
3ZYU_lig 0.3181818181818182
4LYW_lig 0.2727272727272727
4LZR_lig 0.3181818181818182
4LZS_lig 0.3333333333333333
4MR3_lig 0.2727272727272727
4NUC_lig 0.2727272727272727
4NUE_lig 0.2727272727272727
4OGJ_lig 0.2727272727272727
4QB3_lig 0.4090909090909091
4WIV_lig 0.3181818181818182
4XYA_lig 0.4
4Z93_lig 0.4090909090909091
5ACY_lig 0.2727272727272727


In [24]:
print(prot.lm.mcss.MCSSs['3P5O_lig-3U5J_lig'])
prot.load_docking(['3P5O_lig', '3U5J_lig'], 10, load_mcss=True)
print('Reference', prot.lm.mcss.get_rmsd('3P5O_lig', '3U5J_lig', 0, 0))
print('Native', prot.lm.mcss.get_rmsd('3P5O_lig', '3U5J_lig', 0, 3))

print(prot.docking[prot.lm.st].ligands['3P5O_lig'].poses[0].rmsd)
print(prot.docking[prot.lm.st].ligands['3U5J_lig'].poses[0].rmsd)
print(prot.docking[prot.lm.st].ligands['3U5J_lig'].poses[3].rmsd)

3P5O_lig,3U5J_lig,30,22,21,0,c1ccccc1C2=NCc(nnc3C)n3c(c24)cccc4,c1ccccc1C2=NCc(nnc3C)n3c(c24)cccc4,False
Reference 5.249404109966573
Native 0.15945698734917896
0.507117282972
5.12258796291
0.18586569632
